<a href="https://colab.research.google.com/github/pereira-71/-EXAMEN-01-PRIMER-PARCIAL/blob/main/ia2_1_actas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Esto tiene formato de código
```



In [ ]:
#